In [1]:
#Imports
import keras
import tensorflow as tf
from keras import layers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random

#Presets
num_images = 5000

#Import data—data is local
data = pd.read_csv('train.csv')
images = data.iloc[0:num_images,1:]
train_images = images / 255.0
train_images = train_images.values.reshape(num_images, 28, 28, 1)
train_labels = data.iloc[0:num_images,:1]

train_images, validation_images, train_labels, validation_labels = train_test_split(train_images, train_labels, train_size = .2, random_state=42)

class Metadata:
    def __init__(self, dense_layers, dense_layer_nodes, kernels_0, kernels_1):
        self.dense_layers = dense_layers
        self.dense_layer_nodes = dense_layer_nodes
        self.kernels_0 = kernels_0
        self.kernels_1 = kernels_1
        
    def print_metadata(self):
        print(self.dense_layers, self.dense_layer_nodes, self.kernels_0, self.kernels_1)

class ModelWithMetadata:
    def get_accuracy(self, model):
        try:
            model.compile(optimizer=tf.train.AdamOptimizer(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            model.fit(train_images, train_labels, batch_size=100, epochs=10, verbose=0, validation_data=(validation_images, validation_labels))
            return model.evaluate(validation_images, validation_labels)[1]
        except:
            print("Model failed")
            return 0.0
    
    def __init__(self, metadata):
        self.model = self.get_model_from_seed(metadata)
        self.metadata = metadata
        self.accuracy = self.get_accuracy(self.model)
        print("Trained model with accuracy ", self.accuracy)
        self.metadata.print_metadata()
    
    #Returns a model, input is of type metadata
    def get_model_from_seed(self, metadata):
        model = keras.Sequential()
        model.add(keras.layers.Conv2D(metadata.kernels_0, 2, strides=(1,1), padding="valid", input_shape=(28, 28, 1), activation="relu"))
        model.add(keras.layers.AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid',))
        model.add(keras.layers.Conv2D(metadata.kernels_1, 2, strides=(1,1), padding="valid", input_shape=(28, 28, 1), activation="relu"))
        model.add(keras.layers.AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid',))
        model.add(keras.layers.Flatten())
        for i in range(metadata.dense_layers):
            model.add(keras.layers.Dense(metadata.dense_layer_nodes[i], activation="relu", use_bias=True, kernel_initializer="glorot_uniform", bias_initializer="zeros"))
        model.add(keras.layers.Dense(10, activation="softmax", use_bias=True, kernel_initializer="glorot_uniform", bias_initializer="zeros"))
        return model

def mutate(metadata):
    dense_layers = 3
    dense_layer_nodes = list()
    for i in range(dense_layers):
        if (i < len(metadata.dense_layer_nodes)):
                dense_layer_nodes.append(max(10, metadata.dense_layer_nodes[i] + random.randint(-10,10)))
        else:
                dense_layer_nodes.append(random.randint(10,1001))
    kernels_0 = max(10, metadata.kernels_0 + random.randint(-10, 10))
    kernels_1 = max(10, metadata.kernels_1 + random.randint(-10, 10))
    return(Metadata(dense_layers, dense_layer_nodes, kernels_0, kernels_1))

def breed(m1, m2):
        dense_layers = 3
        dense_layer_nodes = m1.dense_layer_nodes if random.getrandbits(1) else m2.dense_layer_nodes
        kernels_0 = m1.kernels_0 if random.getrandbits(1) else m2.kernels_0
        kernels_1 = m1.kernels_1 if random.getrandbits(1) else m2.kernels_1
        return(Metadata(dense_layers, dense_layer_nodes, kernels_0, kernels_1))

Using TensorFlow backend.
/Users/tristanengst/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [ ]:
#Returns a random model
def get_random_model():
    return ModelWithMetadata(Metadata(3, [random.randint(10,1000), random.randint(10,1000), random.randint(10,1000)], random.randint(5,250), random.randint(5,250)))

def at_local_max(x, y, z):
    if z - x < .05:
        return True
    return False

#Assumes there are accuracies 
def evolve(num_generations, models):
    accuracies = []
    need_new_genes = False
    print("Commenced evolution!")
    for i in range(num_generations):
        #Get the accuracy of the generation
        models.sort(key=lambda model: model.accuracy)
        generation_accuracy = 0
        for model in models:
            generation_accuracy += model.accuracy / len(models)
        print("GENERATION:", i, "Best accuracy:", models[len(models) - 1].accuracy, "Average accuracy:", generation_accuracy, "Worst accuracy:", models[0].accuracy)
        models[len(models) - 1].metadata.print_metadata()
        accuracies.append(generation_accuracy)
        x = len(accuracies)
        need_new_genes = models[len(models) - 1].accuracy < .95 or at_local_max(accuracies[x - 3], accuracies[x - 2], accuracies[x - 1])
        
        #Delete all but the best three models and two randomly chosen ones
        saved_1 = models[random.randint(0, len(models) - 1 - 3)]
        saved_2 = models[random.randint(0, len(models) - 1 - 3)]
        while (len(models) > 3):
            models.pop(0)
        models.append(saved_1)
        models.append(saved_2)
        #Add asexual models
        for j in range(1):
            model = ModelWithMetadata(mutate(models[0].metadata))
            models.append(model)
        for j in range(1):
            model = ModelWithMetadata(mutate(models[1].metadata))
            models.append(model)
        for j in range(4):
            model = ModelWithMetadata(mutate(models[2].metadata))
            models.append(model)
        
        #If the best accuracies the model can produce aren't great
        if need_new_genes:
            for j in range(3):
                models.append(get_random_model())
        need_new_genes = False 
        
        #Add sexual models from best 3
        models.sort(reverse=True, key=lambda model: model.accuracy)
        models.append(ModelWithMetadata(breed(models[0].metadata, models[1].metadata)))
        models.append(ModelWithMetadata(breed(models[0].metadata, models[2].metadata)))
        models.append(ModelWithMetadata(breed(models[1].metadata, models[2].metadata)))
        
        #Add sexual models from the best 3 and random
        for i in range(4):
            rand_1 = random.randint(0, 2)
            rand_2 = random.randint(3, len(models) - 1)
            models.append(ModelWithMetadata(breed(models[rand_1].metadata, models[rand_2].metadata)))
    return accuracies

print("This doesn't seem to do anything for a while. Hang with it.")
models = []
for i in range(20):
    models.append(get_random_model())




This doesn't seem to do anything for a while. Hang with it.
4000/4000 [==============================] - 3s 713us/step
Trained model with accuracy  0.90575
3 [706, 927, 435] 76 43
4000/4000 [==============================] - 3s 728us/step
Trained model with accuracy  0.902
3 [948, 21, 95] 71 120
4000/4000 [==============================] - 7s 2ms/step
Trained model with accuracy  0.901
3 [28, 608, 784] 167 147
4000/4000 [==============================] - 5s 1ms/step
Trained model with accuracy  0.90075
3 [969, 66, 531] 141 32
4000/4000 [==============================] - 8s 2ms/step
Trained model with accuracy  0.90275
3 [574, 48, 501] 182 135
4000/4000 [==============================] - 4s 1ms/step
Trained model with accuracy  0.91975
3 [82, 663, 466] 119 128


In [ ]:
accuracies = evolve(150, models)